In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import StandardScaler

In [23]:
data = pd.read_excel(r"D:\user interface\final_data_first_outlier_then_4log.xlsx")

data

data.drop(["Unnamed: 0"], axis=1,inplace=True)

data

,Category,Age,Sex,ALB,ALP,CHE,CHOL,CREA,PROT,Logged ALT,TLogged AST,Logged BIL,DLogged GGT
0,0,32.0,0,38.5,52.50000,6.93,3.23,106.0,69.0,2.041220,0.122195,2.014903,0.913569
1,0,32.0,0,38.5,70.30000,11.17,4.80,74.0,76.5,2.890372,0.152957,1.360977,1.010608
2,0,32.0,0,46.9,74.70000,8.84,5.20,86.0,79.3,3.589059,0.157870,1.808289,1.083867
3,0,32.0,0,43.2,52.00000,7.33,4.74,80.0,75.7,3.421000,0.128548,1.989637,1.085703
4,0,32.0,0,39.2,74.10000,9.15,4.32,76.0,68.7,3.484312,0.154037,2.261763,1.223145
...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,4,62.0,1,32.0,416.60000,5.57,6.30,55.7,68.5,1.774952,0.437121,3.912023,1.868467
611,4,64.0,1,24.0,102.80000,1.54,3.02,63.0,71.3,1.064711,0.287597,2.995732,1.275569
612,4,64.0,1,29.0,87.30000,1.66,3.63,66.7,82.0,1.252763,0.421991,3.871201,1.425996
613,4,46.0,1,33.0,68.28392,3.56,4.20,52.0,71.0,3.663562,0.348954,2.995732,1.364055


Proprosed Data preprocessing upto feature selection step

In [24]:
data = pd.read_csv(r"D:\all python works\for thesis\HepatitisCdata.csv")


data.drop([ "Unnamed: 0"], axis=1, inplace=True)
print(data.columns)

data.replace(to_replace=['0=Blood Donor', '0s=suspect Blood Donor', '1=Hepatitis',
       '2=Fibrosis', '3=Cirrhosis'],
             value=[0,1,2,3,4],inplace=True)

data.replace(to_replace=['m','f'],
             value=[0,1],inplace=True)


               #handling missing values of cirrosis
data_cirrhosis=data[data['Category']== 4]
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(data_cirrhosis.iloc[:,1:13])
data_cirrhosis.iloc[:,1:13]= si.transform(data_cirrhosis.iloc[:,1:13])

               #for blood donor
data_blood_donor=data[data['Category']==0]
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(data_blood_donor.iloc[:,1:13])
data_blood_donor.iloc[:,1:13]= si.transform(data_blood_donor.iloc[:,1:13])#set_index was not used so needed to start from column 3

               #for heaptitis
data_hepatitis=data[data['Category']==2]
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(data_hepatitis.iloc[:,1:13])
data_hepatitis.iloc[:,1:13]= si.transform(data_hepatitis.iloc[:,1:13])#set_index was not used so needed to start from column 3

           #for fibrosis
data_Fibrosis=data[data['Category']==3]
from sklearn.impute import SimpleImputer
si = SimpleImputer(missing_values = np.nan, strategy ='mean')
si = si.fit(data_Fibrosis.iloc[:,1:13])
data_Fibrosis.iloc[:,1:13]= si.transform(data_Fibrosis.iloc[:,1:13])

               #for suspect blood donor
data_suspect=data[data['Category']== 1]

data=pd.concat([data_blood_donor,data_suspect,data_cirrhosis,data_Fibrosis,data_hepatitis])
data.info()

# Assuming data is your dataframe
# First, let's filter only category 0
data_category0 = data[data['Category'] == 0]

# Now, let's detect and replace outliers in each column
new_columns = [ 'Age', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']

outliers_exist = True
while outliers_exist:
    outliers_exist = False
    for column in new_columns:
        Q1 = data_category0[column].quantile(0.25)
        Q3 = data_category0[column].quantile(0.75)
        IQR = Q3 - Q1

        # Define outliers
        outliers = ((data_category0[column] < (Q1 - 1.5 * IQR)) | (data_category0[column] > (Q3 + 1.5 * IQR)))

        if outliers.any():
            outliers_exist = True

            # Replace outliers with the mean of the non-outliers values
            data_category0.loc[outliers, column] = data_category0.loc[~outliers, column].mean()

# Now, replace the original category 0 with the cleaned one
for index in data_category0.index:
    data.loc[index] = data_category0.loc[index]
                                       #now log transform
data['Logged ALT']=np.log(data['ALT'])
data['Logged AST']=np.log(data['AST'])
data['DLogged AST']=np.log(data['Logged AST'])
data['TLogged AST']=np.log(data['DLogged AST']) #Further log transform of AST will result the values to be negative so it will not be logtransformable
data['Logged BIL']=np.log(data['BIL'])
data['Logged GGT']=np.log(data['GGT'])
data['DLogged GGT']=np.log(data['Logged GGT']) #Further log transform of GGT will result the values to be negative so it will not be logtransformable


new_columns=['Age','Sex','ALB','CHE','CHOL','PROT','CREA','ALP','Logged ALT','TLogged AST','Logged BIL','DLogged GGT','Category']
data[new_columns]
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# Set the style to white
sns.set_style("whitegrid")
data.drop(['ALT'], axis=1,inplace=True)
data.drop(['AST'], axis=1,inplace=True)
data.drop(['Logged AST'], axis=1,inplace=True)
data.drop(['DLogged AST'], axis=1,inplace=True)
data.drop(['BIL'], axis=1,inplace=True)
data.drop(['GGT'], axis=1,inplace=True)
data.drop(['Logged GGT'], axis=1,inplace=True)

data

Index(['Category', 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE',
       'CHOL', 'CREA', 'GGT', 'PROT'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 615 entries, 0 to 563
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Category  615 non-null    int64  
 1   Age       615 non-null    int64  
 2   Sex       615 non-null    int64  
 3   ALB       615 non-null    float64
 4   ALP       615 non-null    float64
 5   ALT       615 non-null    float64
 6   AST       615 non-null    float64
 7   BIL       615 non-null    float64
 8   CHE       615 non-null    float64
 9   CHOL      615 non-null    float64
 10  CREA      615 non-null    float64
 11  GGT       615 non-null    float64
 12  PROT      615 non-null    float64
dtypes: float64(10), int64(3)
memory usage: 67.3 KB


C:\Users\ahadk\AppData\Local\Temp\ipykernel_15412\2886193278.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '46.966037735849056' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_category0.loc[outliers, column] = data_category0.loc[~outliers, column].mean()
C:\Users\ahadk\AppData\Local\Temp\ipykernel_15412\2886193278.py:75: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '46.966037735849056' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[index] = data_category0.loc[index]


,Category,Age,Sex,ALB,ALP,CHE,CHOL,CREA,PROT,Logged ALT,TLogged AST,Logged BIL,DLogged GGT
0,0,32.0,0,38.5,52.5,6.93,3.23,106.0,69.0,2.041220,0.122195,2.014903,0.913569
1,0,32.0,0,38.5,70.3,11.17,4.80,74.0,76.5,2.890372,0.152957,1.360977,1.010608
2,0,32.0,0,46.9,74.7,8.84,5.20,86.0,79.3,3.589059,0.157870,1.808289,1.083867
3,0,32.0,0,43.2,52.0,7.33,4.74,80.0,75.7,3.421000,0.128548,1.989637,1.085703
4,0,32.0,0,39.2,74.1,9.15,4.32,76.0,68.7,3.484312,0.154037,2.261763,1.223145
...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2,58.0,0,43.0,99.1,5.95,6.15,147.3,65.6,2.501436,0.352217,2.564949,1.823976
560,2,33.0,1,43.0,29.6,6.88,5.72,58.8,78.2,1.335001,0.034505,1.791759,0.892959
561,2,41.0,1,37.0,31.2,7.08,5.30,60.8,82.4,2.104134,0.257336,1.945910,1.165275
562,2,50.0,1,40.0,32.7,7.51,4.67,56.6,70.1,2.197225,0.294541,2.302585,1.132881


# Find the Feature ranking


In [26]:
X=data.drop(columns=['Category'],axis=1)
y=data['Category']
bestfeatures = SelectKBest(score_func=f_classif, k='all')
fit = bestfeatures.fit(X, y)

# F-scores and p-values
f_scores = fit.scores_
p_values = fit.pvalues_

# Create DataFrames for scores and p-values
dfscores = pd.DataFrame({'F-score': f_scores, 'p-value': p_values})
dfcolumns = pd.DataFrame(X.columns)

# Combine DataFrames
scorefeature = pd.concat([dfcolumns, dfscores], axis=1)
scorefeature.columns = ['specs', 'F-score', 'p-value']

# Display top 12 features based on F-score
top_features = scorefeature.nlargest(12, 'F-score')
print(top_features)

          specs     F-score       p-value
9   TLogged AST  164.683024  1.585421e-95
10   Logged BIL  144.831239  5.417167e-87
11  DLogged GGT  133.944251  4.542415e-82
2           ALB   69.390666  2.036520e-48
4           CHE   44.959085  4.213372e-33
7          PROT   36.281990  3.196749e-27
3           ALP   31.173246  1.218003e-23
8    Logged ALT   25.730177  1.003550e-19
5          CHOL   19.526131  3.914646e-15
0           Age   11.694165  3.712404e-09
6          CREA   11.403133  6.236281e-09
1           Sex    1.918366  1.057726e-01


Similaly perfrom the CHi2 Square test by changing the function in the above code